# Основные положения

## Виды обучения

В зависимости от доступного нам объема данных и их вида, мы можем применить два разных метода обучения.

### Обучение с подкреплением (supervised learning)

## 2D и 3D представления

Мы можем рассматривать наши томограммы в двух размерностях: 2D и 3D.

### 2D

* Каждое сечение 3D томографического изображения рассматривается отдельно как двумерная картинка;
* Обучение выполняется на множестве 2D срезов, где каждая пара LR-HR создаётся отдельно;
* Используются обычные 2D-свёртки (Conv2D) и стандартные архитектуры CNN или GAN;

**Преимущества**:

* Меньше требований к видеопамяти;
* Проще реализовать;
* Быстро обучается;
* Подходит, если структура пор не требует учёта межслойных связей.

**Недостатки**:

* Не учитываются пространственные корреляции между срезами (по оси z);
* Модель видит только «плоские» текстуры.

Пример: в статье Buono et al. (2023) сначала использовались 2D U-Net и 2D pix2pix, которые дали наилучшие результаты по метрикам (PSNR ≈ 28.8 дБ, SSIM ≈ 0.82).

### 3D

* Входные данные — трёхмерные блоки (patches), например 64×64×64 вокселей;
* Сети используют 3D-свёртки (Conv3D), обучаясь восстанавливать объёмные структуры и связи между слоями;
* Применяется в задачах, где важна непрерывность порового пространства, микротрещины, каналы и т.п.

**Преимущества**:

* Сохраняются пространственные зависимости между срезами;
* Точнее восстанавливается геометрия пор.

**Недостатки**:

* Гораздо больше памяти и времени на обучение;
* Требуется крупный датасет 3D-томов;
* Труднее нормализовать и аугментировать данные.

Пример: в той же работе Buono et al. тестировались 3D U-Net и 3D pix2pix, но они показали худшие результаты (SSIM ≈ 0.68–0.70) именно из-за ограничений по числу 3D-патчей и большому количеству параметров.

## Архитектуры

Существуют три основных архитектуры нейронных сетей для решения задачи superresolution. Каждая из них обладает своими преимуществами и недостатками.

### CNN

CNN (Convolutional Neural Network) - архитектура НС, использующая сверточные слои для получения признаков.

**Преимущества**:

1. Эффективно улавливают локальные паттерны и структуры;
2. Хорошо генерализуют на больших датасетах.

**Недостатки**:

1. Требуют пар LR и HR (supervised learning);
2. Могут иметь проблемы с улавливанием признаков на дистанции.

Возможные архитектуры: SRCNN, RCAN, U-Net.

### GAN

**GAN** (Generative Adversarial Network) - две нейросети, где одна выступает в роли генератора (G), а другая - в роли дискриминатора (D). G пытается сгенерировать максимально правдоподобное изображение, а D - отличить настоящее от сгенерированного.

**Преимущества**:
1. Высокая реалистичность изображений на выходе;
2. Высокое разнообразие выходных изображений;
3. Требует наличие лишь одного изображения HR для создания SR (unsupervised learning), что позволяет сократить количество требуемых данных.

**Недостатки**:
1. Относительно нестабильное обучение по сравнению с другими архитектурами;
2. Mode collapse;
3. Требует высоких вычислительных мощностей.

Возможные архитектуры: ESRGAN, Real-ESRGAN, CycleGAN.

### Diffusion Model

**Diffusion Model** - вероятностная архитектура, основанная на итеративном зашумлении изображения, а затем его восстановлении. 

**Преимущества**:

1. Также как и у GAN, высокое качество выходных изображений;
2. Устойчивость к шуму в данных;

**Недостатки**:
1. Требует высоких вычислительных мощностей;
2. Процесс обучения и инференса может требовать сложных процедур.

Возможные архитектуры: Shortcut Sampling for Diffusion (SSD), Denoising Diffusion Null-Space Model (DDNM)

## Метрики качества

В ходе исследований были найдены две основные метрики качества для задач SR: **PSNR** и **SSIM**.

### PSNR

Метрика **PSNR** (Peak Signal-to-Noise Ratio) — это количественная мера качества восстановленного изображения по сравнению с эталонным (HR). Она показывает, насколько сильно зашумлено или искажено SR-изображение.

Формула:

#### $PSNR = 10*log_{10}(\frac{MAX^2_{i}}{MSE})$,

где:
* $MAX_{i}$ - максимальное возможное значение интенсивности пикселя (обычно 255 для 8-битных изображений),
* $MSE$ - среднеквадратичная ошибка между восстановленным изображением и эталоном:

#### $MSE=\frac{1}{H * W}\sum^{H}_{𝑖=0}\sum^{W}_{𝑗=0}{(I_{HR}(𝑖,𝑗)−I_{LR}(𝑖,𝑗))^2}$

Интерпретация:

* Выше PSNR - лучше качество (меньше искажений);
* Типичные значения:
    * < 25 дБ — слабое качество (заметные артефакты),
    * 25 – 30 дБ — приемлемое качество,
    * 30 дБ — хорошее соответствие HR.

### SSIM

Метрика SSIM (Structural Similarity Index Measure) моделирует человеческое восприятие, сравнивая локальные участки двух изображений по трём компонентам:
* Luminance (яркость) — насколько совпадает средний уровень освещённости;
* Contrast (контраст) — насколько совпадает разброс интенсивностей;
* Structure (структура) — насколько совпадает форма сигналов (узоры, текстуры).

Формула для двух окон изображений $x$ и $y$:

#### $SSIM(x, y)=\frac{(2𝜇_{x}𝜇_{y}+C_{1})(2𝜎_{x}𝜎_{y}+𝐶_{2})}{(𝜇^2_{x}+𝜇^2_{y}+𝐶_{1})(𝜎^2_{x}+𝜎^2_{y}+𝐶_{2})}$,
где:
* $𝜇_{x}$, $𝜇_{y}$ - средние яркости окон,
* $𝜎^2_{x}$, $𝜎^2_{y}$ - дисперсии (контраст),
* $𝜎_{x}$, $𝜎_{y}$ - стандартные отклонения, их произведение $𝜎_{x}𝜎_{y}$ является ковариацией (сходство структуры),
* $C_{1}$, $C_{2}$ - маленькие константы для численной стабильности (зависят от динамического диапазона, например $C_{1} = (0.01L)^2$, $C_{2} = (0.03L)^2$, где $L = 255$.)

Интерпертация частей формулы:

* Яркость:
  #### $l(x, y) = \frac{2𝜇_{x}𝜇_{y} + C_{1}}{𝜇^2_{x} + 𝜇^2_{y} + C_{1}}$,
  показывает, насколько средняя яркость совпадает.
  
* Контраст:
  #### $c(x, y) = \frac{2𝜎_{x}𝜎_{y} + C_{2}}{𝜎^2_{x} + 𝜎^2_{y} + C_{2}}$,

  оценивает, совпадает ли разброс значений (контраст).

* Структура:
  #### $s(x, y) = \frac{𝜎_{xy} + C_{3}}{𝜎_{x}𝜎_{y} + C_{3}}$,

  проверяет, похожи ли формы сигналов (т.е. корреляция).


Комбинируются они как:
#### $SSIM = l(x, y)^{α} * c(x, y)^{β} * s(x, y)^{γ}$
обычно $α=β=γ=1$.

Интерпретация:
* $SSIM = 1$ - изображения идентичны;
* $SSIM = 0$ - нет структурного сходства;
* Возможны отрицательные значения (сильное различие).

В отличие от $PSNR$, $SSIM$ чувствителен к структуре, поэтому, например:
* Размытие (потеря текстуры) сильно снижает $SSIM$,
* Сдвиг яркости на константу почти не влияет.